In [1]:
# This code come from:
# https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb

In [2]:
# Trump speeches from: https://github.com/ryanmcdermott/trump-speeches

In [3]:
import unidecode
import string
import random
import re
import torch
import torch.nn as nn
from torch.autograd import Variable
import time
import math

In [4]:
use_gpu = True

In [5]:
torch.cuda.is_available()

True

In [6]:
all_characters = string.printable
n_characters = len(all_characters)

In [7]:
file = unidecode.unidecode(open('trumpSpeeches.txt').read())
file_len = len(file)
print('file_len = ' + str(file_len))

file_len = 896987


In [8]:
chunk_len = 200

def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

l immigration. They wouldn't even be talking about it. They wouldn't even be talking about it.


And we're going to build a wall and we're going to get, you know, the gang members in Baltimore and in C


In [9]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [10]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)

print(char_tensor('abcDEF'))

tensor([10, 11, 12, 39, 40, 41])


In [11]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    if use_gpu:
        return inp.cuda(), target.cuda()
    return inp, target

In [12]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    if use_gpu:
        prime_input = prime_input.cuda()
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)

    return predicted

In [13]:
def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [20]:
def train(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        print(inp[c])
        output, hidden = decoder(inp[c], hidden)
        loss += criterion(output, target[c].view(1))

    loss.backward()
    decoder_optimizer.step()

    return loss.data[0] / chunk_len

In [15]:
n_epochs = 5000
print_every = 100
plot_every = 10
hidden_size = 200
n_layers = 2
lr = 0.0005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)

In [16]:
decoder = decoder.cuda()

In [17]:
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [21]:
loss = train(*random_training_set()) 

RuntimeError: cuda runtime error (77) : an illegal memory access was encountered at /pytorch/aten/src/THC/generic/THCTensorCopy.cpp:20

In [14]:
n_epochs = 5000
print_every = 100
plot_every = 10
hidden_size = 200
n_layers = 2
lr = 0.0005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder = decoder.cuda()
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    loss = train(*random_training_set())       
    loss_avg += loss

#     if epoch % print_every == 0:
#         print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
#         print(evaluate('Wh', 100), '\n')

#     if epoch % plot_every == 0:
#         all_losses.append(loss_avg / plot_every)
#         loss_avg = 0

RuntimeError: cuda runtime error (30) : unknown error at /pytorch/aten/src/THC/THCGeneral.cpp:74

In [ ]:

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)


In [ ]:
decoder

In [ ]:
a = decoder.cuda()

In [ ]:
a